# LangGraph Deep Agent with MCP Tools, Memory, and RAG

This notebook demonstrates how to build a powerful LangGraph agent with:
- **MCP Tools**: Integration with Model Context Protocol servers for external tools
- **Memory**: Thread-scoped conversation persistence using checkpointers
- **RAG**: Retrieval Augmented Generation for knowledge base queries
- **Gemini 2.5 Flash**: Using Google's latest LLM model

## Architecture
```
+------------------+     +------------------+     +------------------+
|    MCP Tools     |---->|   Deep Agent     |---->|   RAG Retriever  |
|  (SAP Docs, etc) |     | (Gemini 2.5)     |     |  (Vector Store)  |
+------------------+     +------------------+     +------------------+
                               |
                               v
                    +------------------+
                    |   Checkpointer   |
                    |    (Memory)      |
                    +------------------+
```

## 1. Install Dependencies

First, ensure you have the required packages installed:

In [ ]:
# Uncomment and run if packages are not installed
# !pip install langchain langgraph langchain-google-genai langchain-mcp-adapters deepagents
# !pip install langchain-core langchain-community langchain-text-splitters
# !pip install langchain-huggingface sentence-transformers  # HuggingFace embeddings (local, no API key)
# !pip install beautifulsoup4 lxml  # Required for WebBaseLoader

## 2. Imports and Setup

In [1]:
import os
import asyncio
from typing import List, Dict, Any

# LangChain core
from langchain_core.documents import Document
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.tools import tool

# LangGraph
from langgraph.checkpoint.memory import MemorySaver

# MCP Adapters
from langchain_mcp_adapters.client import MultiServerMCPClient

# Gemini LLM
from langchain_google_genai import ChatGoogleGenerativeAI

# HuggingFace Embeddings (local, no API key needed)
from langchain_huggingface import HuggingFaceEmbeddings

# Deep Agents
from deepagents import create_deep_agent

print('[OK] All imports successful!')

[OK] All imports successful!


## 3. Environment Configuration

Set up API keys for Gemini and any other services:

In [2]:
# Verify API key is set
# You can set it here or use environment variables
# os.environ['GOOGLE_API_KEY'] = 'your-api-key-here'

api_key = os.environ.get('GEMINI_API_KEY') or os.environ.get('GOOGLE_API_KEY')
if not api_key:
    print('[WARNING] GEMINI_API_KEY or GOOGLE_API_KEY not found in environment')
    print('Please set your API key using: os.environ["GOOGLE_API_KEY"] = "your-key"')
else:
    print('[OK] API key found')

[OK] API key found


## 4. Initialize Gemini 2.5 Flash LLM

Create the LLM instance with temperature=0 for deterministic responses:

In [3]:
# Initialize Gemini 2.5 Flash with temperature=0 for deterministic responses
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,  # CRITICAL: Always 0 for consistency
    max_tokens=None,
    timeout=None,
    max_retries=2,
    convert_system_message_to_human=True  # Required for Gemini compatibility
)

print(f'[OK] LLM initialized: {llm.model}')

[OK] LLM initialized: models/gemini-2.5-flash


## 5. RAG Setup - Fetch Knowledge Base from Websites

We'll fetch real documentation about MCP and Deep Agents from their official sources to build a knowledge base.

In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# URLs to fetch documentation from
doc_urls = [
    "https://modelcontextprotocol.io/docs/getting-started/intro",
    "https://modelcontextprotocol.io/docs/learn/architecture",
    "https://docs.langchain.com/oss/python/deepagents/overview",
    "https://docs.langchain.com/oss/python/deepagents/middleware",
    "https://docs.langchain.com/oss/python/langchain/mcp",
]

print("[...] Fetching documentation from websites using LangChain WebBaseLoader...")

# Fetch documents using LangChain's WebBaseLoader
try:
    loader = WebBaseLoader(
        web_paths=doc_urls,
        bs_kwargs={"parse_only": None},  # Parse full page
    )
    raw_docs = loader.load()
    print(f"[OK] Loaded {len(raw_docs)} raw documents from web")
    
    # Split documents into smaller chunks for better retrieval
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1500,
        chunk_overlap=200,
        separators=["\n\n", "\n", ". ", " ", ""]
    )
    documents = text_splitter.split_documents(raw_docs)
    print(f"[OK] Split into {len(documents)} chunks")
    
except Exception as e:
    print(f"[WARNING] Failed to fetch some URLs: {e}")
    documents = []

# Add some curated content as backup/supplement
curated_documents = [
    Document(
        page_content="""MCP (Model Context Protocol) is an open-source standard for connecting AI applications to external systems. 
        Using MCP, AI applications like Claude or ChatGPT can connect to data sources (e.g. local files, databases), 
        tools (e.g. search engines, calculators) and workflows (e.g. specialized prompts)—enabling them to access key 
        information and perform tasks. Think of MCP like a USB-C port for AI applications. Just as USB-C provides a 
        standardized way to connect electronic devices, MCP provides a standardized way to connect AI applications 
        to external systems. MCP follows a client-server architecture where an MCP host establishes connections to 
        one or more MCP servers. The key participants are: MCP Host (the AI application), MCP Client (maintains 
        connection to server), and MCP Server (provides context to clients).""",
        metadata={"source": "mcp_curated", "topic": "overview", "url": "https://modelcontextprotocol.io"}
    ),
    Document(
        page_content="""Deep agents are built with a modular middleware architecture. Deep agents have access to:
        A planning tool (write_todos) - enables agents to break down complex tasks into discrete steps, track progress
        A filesystem for storing context and long-term memories (ls, read_file, write_file, edit_file)
        The ability to spawn subagents (task tool) - creates ephemeral agents for isolated multi-step tasks
        Each feature is implemented as separate middleware. When you create a deep agent with create_deep_agent, 
        TodoListMiddleware, FilesystemMiddleware, and SubAgentMiddleware are automatically attached.
        Middleware is composable—you can add as many or as few middleware to an agent as needed.""",
        metadata={"source": "deepagents_curated", "topic": "middleware", "url": "https://docs.langchain.com/oss/python/deepagents/middleware"}
    ),
    Document(
        page_content="""deepagents is a standalone library for building agents that can tackle complex, multi-step tasks.
        Built on LangGraph and inspired by applications like Claude Code, Deep Research, and Manus, deep agents come 
        with planning capabilities, file systems for context management, and the ability to spawn subagents.
        Core capabilities include: Planning and task decomposition with write_todos tool, Context management with 
        file system tools preventing context window overflow, Subagent spawning for context isolation and parallel 
        execution, and Long-term memory using LangGraph's Store for persistent memory across threads.""",
        metadata={"source": "deepagents_curated", "topic": "overview", "url": "https://docs.langchain.com/oss/python/deepagents/overview"}
    ),
    Document(
        page_content="""Model Context Protocol (MCP) is an open protocol that standardizes how applications provide 
        tools and context to LLMs. LangChain agents can use tools defined on MCP servers using the 
        langchain-mcp-adapters library. MultiServerMCPClient enables agents to use tools defined across one or 
        more MCP servers. MCP servers can use different transports: stdio for local Python scripts, 
        streamable_http for remote HTTP servers. The MCP client is stateless by default - each tool invocation 
        creates a fresh MCP ClientSession, executes the tool, and then cleans up.""",
        metadata={"source": "langchain_mcp_curated", "topic": "integration", "url": "https://docs.langchain.com/oss/python/langchain/mcp"}
    ),
]

# Combine fetched and curated documents
all_documents = documents + curated_documents

print(f'\n[OK] Created knowledge base with {len(all_documents)} documents:')
print(f'  - {len(documents)} fetched from websites')
print(f'  - {len(curated_documents)} curated documents')

USER_AGENT environment variable not set, consider setting it to identify your requests.


[...] Fetching documentation from websites using LangChain WebBaseLoader...
[OK] Loaded 5 raw documents from web
[OK] Split into 57 chunks

[OK] Created knowledge base with 61 documents:
  - 57 fetched from websites
  - 4 curated documents


## 6. Create Vector Store and Retriever

Initialize the vector store with HuggingFace's `all-MiniLM-L6-v2` embedding model (local, no API key needed):

In [5]:
# Initialize HuggingFace embeddings (local, no API key required)
# Using all-MiniLM-L6-v2: Fast, lightweight, 384 dimensions
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={
        'device': 'cpu',           # Works on any machine
        'trust_remote_code': False  # Security
    },
    encode_kwargs={
        'normalize_embeddings': True,  # Better similarity scores
        'batch_size': 32               # Optimize for speed
    }
)
print('[OK] HuggingFace embeddings initialized (all-MiniLM-L6-v2)')

# Create in-memory vector store from fetched documents
vectorstore = InMemoryVectorStore.from_documents(
    documents=all_documents,
    embedding=embeddings
)

# Create retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

print(f'[OK] Vector store created with {len(all_documents)} documents')
print('[OK] Retriever configured to return top 3 results')

[OK] HuggingFace embeddings initialized (all-MiniLM-L6-v2)
[OK] Vector store created with 61 documents
[OK] Retriever configured to return top 3 results


## 7. Create RAG Retriever Tool

Wrap the retriever as a tool that the agent can use:

In [6]:
@tool
def search_knowledge_base(query: str) -> str:
    """
    Search the internal knowledge base for information about LangGraph, 
    Deep Agents, MCP, memory, and RAG concepts.
    
    Args:
        query: The search query to find relevant documents
        
    Returns:
        Relevant document excerpts from the knowledge base
    """
    docs = retriever.invoke(query)
    if not docs:
        return "No relevant documents found in the knowledge base."
    
    # Format results with source metadata
    results = []
    for i, doc in enumerate(docs, 1):
        source = doc.metadata.get('source', 'unknown')
        topic = doc.metadata.get('topic', 'general')
        results.append(f"[{i}] Source: {source} | Topic: {topic}\n{doc.page_content}")
    
    return "\n\n".join(results)


# Test the RAG tool
test_result = search_knowledge_base.invoke({"query": "What is LangGraph?"})
print('[OK] RAG tool created and tested')
print('\nTest query result:')
print(test_result[:500] + '...' if len(test_result) > 500 else test_result)

[OK] RAG tool created and tested

Test query result:
[1] Source: https://modelcontextprotocol.io/docs/learn/architecture | Topic: general
For specific implementation details, please refer to the documentation for your language-specific SDK.
​Scope
The Model Context Protocol includes the following projects:

[2] Source: deepagents_curated | Topic: overview
deepagents is a standalone library for building agents that can tackle complex, multi-step tasks.
        Built on LangGraph and inspired by applications like Claude Code, Deep Research, and Manu...


## 8. MCP Tools Integration

Set up the MCP client to connect to external tool servers. The MCP client can connect to multiple servers simultaneously.

In [8]:
async def get_mcp_tools():
    """
    Initialize MCP client and get tools from configured servers.
    
    MCP Servers can be:
    - stdio: Local Python scripts
    - streamable_http: Remote HTTP servers
    
    Returns:
        List of LangChain tools from MCP servers
    """
    client = MultiServerMCPClient(
        {
            # SAP Documentation MCP Server
            # "sap_docs": {
            #     "transport": "streamable_http",
            #     "url": "https://mcp-sap-docs.marianzeis.de/mcp",
            # },
            # You can add more MCP servers here:
            # "math": {
            #     "transport": "stdio",
            #     "command": r"C:\App\Anaconda\python.exe",
            #     "args": [r"path\to\math_server.py"],
            # },
            "weather": {
                "transport": "streamable_http",
                "url": "http://localhost:8000/mcp",
            },
        }
    )
    
    tools = await client.get_tools()
    return tools


# Get MCP tools
mcp_tools = await get_mcp_tools()
print(f'[OK] Loaded {len(mcp_tools)} MCP tools:')
for t in mcp_tools:
    print(f'  - {t.name}')

[OK] Loaded 1 MCP tools:
  - get_weather


## 9. Memory/Checkpointer Setup

Set up the checkpointer for conversation memory. This enables:
- Short-term memory within a conversation thread
- Ability to resume conversations
- Multi-turn context retention

In [9]:
# Create checkpointer for memory persistence
# InMemorySaver: For development/testing (data lost on restart)
# PostgresSaver: For production (persistent across restarts)
checkpointer = MemorySaver()

print('[OK] Checkpointer initialized (InMemorySaver)')
print('\nMemory types available:')
print('  - Short-term: Conversation history within a thread')
print('  - Long-term: Use StoreBackend for cross-thread persistence')

[OK] Checkpointer initialized (InMemorySaver)

Memory types available:
  - Short-term: Conversation history within a thread
  - Long-term: Use StoreBackend for cross-thread persistence


## 10. Create the Deep Agent

Combine all components into a powerful Deep Agent:
- **LLM**: Gemini 2.5 Flash
- **Tools**: RAG retriever + MCP tools
- **Memory**: Checkpointer for conversation persistence
- **Middleware**: TodoList, Filesystem, SubAgent (included by default)

In [10]:
# Combine all tools: RAG + MCP
all_tools = [search_knowledge_base] + mcp_tools

# System prompt for the agent
system_prompt = """\
You are an intelligent research assistant with access to multiple knowledge sources.

Your capabilities:
1. **Internal Knowledge Base (RAG)**: Use 'search_knowledge_base' to query information about 
   LangGraph, Deep Agents, MCP, memory systems, and RAG concepts.

2. **SAP Documentation (MCP)**: Use SAP documentation tools to search for ABAP, UI5, CAP, 
   and other SAP-related information.

3. **Planning**: Break down complex tasks into manageable steps using the todo list.

4. **File System**: Store important findings and notes for later reference.

Guidelines:
- Always search relevant knowledge sources before answering technical questions
- Cite your sources when providing information
- For complex questions, create a plan first
- Be concise but thorough in your responses
"""

# Create the deep agent with all components
agent = create_deep_agent(
    model=llm,
    tools=all_tools,
    system_prompt=system_prompt,
    checkpointer=checkpointer,
)

print('[OK] Deep Agent created with:')
print(f'  - LLM: Gemini 2.5 Flash')
print(f'  - Tools: {len(all_tools)} ({len(mcp_tools)} MCP + 1 RAG)')
print(f'  - Memory: MemorySaver checkpointer')
print(f'  - Middleware: TodoList, Filesystem, SubAgent')

[OK] Deep Agent created with:
  - LLM: Gemini 2.5 Flash
  - Tools: 2 (1 MCP + 1 RAG)
  - Memory: MemorySaver checkpointer
  - Middleware: TodoList, Filesystem, SubAgent


## 11. Helper Function for Agent Interaction

Create a helper to easily chat with the agent:

In [13]:
from IPython.display import display, Markdown


def extract_text_content(content) -> str:
    """
    Extract text from various content formats.
    
    Handles:
    - Plain strings
    - List of content blocks (e.g., [{"type": "text", "text": "..."}])
    - AIMessage with content attribute
    """
    if content is None:
        return ""
    
    # If it's already a string, return as-is
    if isinstance(content, str):
        return content
    
    # If it's a list of content blocks (common with Gemini/Claude)
    if isinstance(content, list):
        text_parts = []
        for block in content:
            if isinstance(block, str):
                text_parts.append(block)
            elif isinstance(block, dict):
                # Handle {"type": "text", "text": "..."} format
                if block.get("type") == "text":
                    text_parts.append(block.get("text", ""))
                # Handle other dict formats
                elif "text" in block:
                    text_parts.append(block["text"])
                elif "content" in block:
                    text_parts.append(str(block["content"]))
            else:
                # Try to convert to string
                text_parts.append(str(block))
        return "\n".join(text_parts)
    
    # Fallback: convert to string
    return str(content)


async def chat(message: str, thread_id: str = "default") -> str:
    """
    Send a message to the agent and get a response.
    
    Args:
        message: The user's message
        thread_id: Conversation thread ID for memory continuity
        
    Returns:
        The agent's response text
    """
    response = await agent.ainvoke(
        {"messages": [{"role": "user", "content": message}]},
        {"configurable": {"thread_id": thread_id}}
    )
    
    # Extract the last message content
    last_message = response["messages"][-1]
    
    # Handle various content formats
    return extract_text_content(last_message.content)


def display_response(response):
    """Display the response as formatted markdown."""
    # Ensure response is a string
    text = extract_text_content(response) if not isinstance(response, str) else response
    
    if text:
        display(Markdown(text))
    else:
        print("[No response content]")


print('[OK] Helper functions defined')

[OK] Helper functions defined


## 12. Example Usage

### Example 1: RAG Query (Internal Knowledge Base)

In [33]:
# Query the internal knowledge base using RAG
response1 = await chat(
    "What is the difference between short-term and long-term memory in LangGraph?",
    # "How is the weather in Sao Paulo today?",
    thread_id="rag-demo"
)
display_response(response1)

In LangGraph, particularly within the context of Deep Agents, the distinction between short-term and long-term memory is as follows:

*   **Short-term memory** refers to the default filesystem used by tools like `ls`, `read_file`, `write_file`, and `edit_file`. This memory is local to the current graph state and is not persistent across different threads or conversations. Information stored here is lost once the agent's current execution or thread concludes. (Source: [2])

*   **Long-term memory** enables agents to retain information persistently across multiple conversations and threads. This is achieved by configuring a `CompositeBackend` to route specific file paths (e.g., `/memories/`) to a `StoreBackend`, which then utilizes a `Store` (such as `InMemoryStore`). This setup allows agents to save and retrieve information from past interactions, ensuring data availability beyond the current session. (Source: [1], [2])

### Example 2: MCP Query (SAP Documentation)

In [ ]:
# Query SAP documentation using MCP tools
response2 = await chat(
    "How do I handle internal tables in modern ABAP? Give me a brief overview.",
    thread_id="mcp-demo"
)
display_response(response2)

### Example 3: Memory Continuity Demo

Demonstrate that the agent remembers previous messages within a thread:

In [23]:
# First message in a new thread
response3a = await chat(
    "My name is Alice and I'm learning about AI agents.",
    thread_id="memory-demo"
)
print("First response:")
display_response(response3a)

First response:


Hello Alice! It's great to meet you. I can help you learn about AI agents. What specifically about AI agents are you interested in? Are there any particular concepts or aspects you'd like to explore first?

In [24]:
# Follow-up message in the same thread - agent should remember the name
response3b = await chat(
    "What's my name? And what was I learning about?",
    thread_id="memory-demo"  # Same thread ID!
)
print("Follow-up response (should remember context):")
display_response(response3b)

Follow-up response (should remember context):


Your name is Alice, and you were learning about AI agents.

### Example 4: Combined Query (RAG + MCP)

The agent can use multiple tools in one query:

In [ ]:
# Combined query that might use both RAG and MCP
response4 = await chat(
    "First, explain what Deep Agents middleware provides (use internal knowledge), "
    "then give me a quick tip about clean ABAP coding (use SAP docs).",
    thread_id="combined-demo"
)
display_response(response4)

## 13. Direct Tool Testing

Test tools directly without the agent:

In [25]:
# Test RAG tool directly
print("=== RAG Tool Test ===")
rag_result = search_knowledge_base.invoke({"query": "What is MCP protocol?"})
print(rag_result)
print()

# Test MCP tool directly (if available)
if mcp_tools:
    print("=== MCP Tool Test (search) ===")
    # Find the search tool
    search_tool = next((t for t in mcp_tools if t.name == 'search'), None)
    if search_tool:
        mcp_result = await search_tool.ainvoke({"query": "ABAP inline declarations"})
        # Print first 1000 chars of result
        print(str(mcp_result)[:1000] + '...' if len(str(mcp_result)) > 1000 else mcp_result)

=== RAG Tool Test ===
[1] Source: langchain_mcp_curated | Topic: integration
Model Context Protocol (MCP) is an open protocol that standardizes how applications provide 
        tools and context to LLMs. LangChain agents can use tools defined on MCP servers using the 
        langchain-mcp-adapters library. MultiServerMCPClient enables agents to use tools defined across one or 
        more MCP servers. MCP servers can use different transports: stdio for local Python scripts, 
        streamable_http for remote HTTP servers. The MCP client is stateless by default - each tool invocation 
        creates a fresh MCP ClientSession, executes the tool, and then cleans up.

[2] Source: mcp_curated | Topic: overview
MCP (Model Context Protocol) is an open-source standard for connecting AI applications to external systems. 
        Using MCP, AI applications like Claude or ChatGPT can connect to data sources (e.g. local files, databases), 
        tools (e.g. search engines, calculators) and 

## 14. Configuration Options

### Additional MCP Servers

You can add more MCP servers to expand the agent's capabilities:

In [ ]:
# Example: Adding more MCP servers
example_mcp_config = """
# MCP Server Configuration Examples:

mcp_servers = {
    # Local stdio server (Python script)
    "math": {
        "transport": "stdio",
        "command": r"C:\\App\\Anaconda\\python.exe",
        "args": ["math_server.py"],
    },
    
    # Remote HTTP server
    "weather": {
        "transport": "streamable_http",
        "url": "https://weather-mcp.example.com/mcp",
    },
    
    # Server with authentication
    "github": {
        "transport": "streamable_http",
        "url": "https://github-mcp.example.com/mcp",
        "headers": {
            "Authorization": "Bearer YOUR_TOKEN"
        }
    },
    
    # FastMCP server
    "fastmcp": {
        "transport": "streamable_http",
        "url": "https://gofastmcp.com/mcp",
    },
}
"""
print(example_mcp_config)

## 15. Enhanced Agent with Long-Term Memory

Now let's upgrade the agent with:
1. **CompositeBackend**: Hybrid storage with ephemeral + persistent paths
2. **StoreBackend**: Cross-thread persistent memory for `/memories/` path
3. **StateBackend**: Ephemeral storage for temporary files

This enables the agent to:
- Save important learnings to `/memories/` that persist forever
- Use temporary workspace files that are cleaned up per-thread


In [14]:
# Import backends for long-term memory
from deepagents.backends import CompositeBackend, StateBackend, StoreBackend
from langgraph.store.memory import InMemoryStore

# Create the persistent store for cross-thread memory
persistent_store = InMemoryStore()

# Define the composite backend factory
def make_hybrid_backend(runtime):
    """
    Create a hybrid backend that:
    - Routes /memories/* to persistent StoreBackend
    - Routes everything else to ephemeral StateBackend
    """
    return CompositeBackend(
        default=StateBackend(runtime),  # Ephemeral storage (per-thread)
        routes={
            "/memories/": StoreBackend(runtime)  # Persistent storage (cross-thread)
        }
    )

# Enhanced system prompt that teaches the agent about memory management
enhanced_system_prompt = """\
You are an intelligent research assistant with access to multiple knowledge sources and persistent memory.

Your capabilities:
1. **Internal Knowledge Base (RAG)**: Use 'search_knowledge_base' to query information about 
   LangGraph, Deep Agents, MCP, memory systems, and RAG concepts.

2. **External Tools (MCP)**: Use available MCP tools to search external documentation.

3. **Planning**: Break down complex tasks into manageable steps using the todo list.

4. **File System with Memory**:
   - **Persistent memories** (/memories/): Write important learnings, user preferences, 
     and knowledge you want to remember across ALL conversations here.
     Example: write_file("/memories/user_preferences.md", "User prefers concise answers...")
   
   - **Temporary workspace** (/workspace/): Use for scratch files and temporary notes.
     These are cleared when the conversation ends.

Guidelines:
- Always search relevant knowledge sources before answering technical questions
- Cite your sources when providing information
- For complex questions, create a plan first
- **Important**: Save valuable learnings to /memories/ so you remember them in future conversations
- Be concise but thorough in your responses
"""

# Create the enhanced deep agent with long-term memory
enhanced_agent = create_deep_agent(
    model=llm,
    tools=all_tools,
    system_prompt=enhanced_system_prompt,
    checkpointer=checkpointer,  # For short-term thread memory
    store=persistent_store,     # For long-term cross-thread memory
    backend=make_hybrid_backend # Hybrid filesystem backend
)

print('[OK] Enhanced Deep Agent created with long-term memory:')
print('  - /memories/* -> Persistent (survives across threads)')
print('  - /workspace/* -> Ephemeral (per-thread only)')
print('  - Store: InMemoryStore (use PostgresStore for production)')


[OK] Enhanced Deep Agent created with long-term memory:
  - /memories/* -> Persistent (survives across threads)
  - /workspace/* -> Ephemeral (per-thread only)
  - Store: InMemoryStore (use PostgresStore for production)


### Demo: Long-Term Memory Persistence

Let's demonstrate how memories persist across different conversation threads:


In [15]:
# Helper function for the enhanced agent
async def chat_enhanced(message: str, thread_id: str = "default") -> str:
    """Chat with the enhanced agent that has long-term memory."""
    response = await enhanced_agent.ainvoke(
        {"messages": [{"role": "user", "content": message}]},
        {"configurable": {"thread_id": thread_id}}
    )
    last_message = response["messages"][-1]
    return extract_text_content(last_message.content)

# Step 1: First conversation - teach the agent something
print("=== Thread 1: Teaching the agent ===")
response1 = await chat_enhanced(
    "My name is Alice and I'm a senior Python developer. "
    "I prefer concise, technical answers with code examples. "
    "Please remember this for our future conversations by saving it to your memories.",
    thread_id="thread-1"
)
display_response(response1)


=== Thread 1: Teaching the agent ===


Hello Alice! I've saved your preferences to my memory. I'll do my best to provide concise, technical answers with code examples in our future conversations. How can I help you today?

In [16]:
# Step 2: NEW thread - test if memory persists
print("=== Thread 2: Testing memory recall (NEW conversation) ===")
response2 = await chat_enhanced(
    "What do you remember about me from your memories? "
    "Check your /memories/ folder.",
    thread_id="thread-2"  # Different thread!
)
display_response(response2)

print("\n--- Memory persistence verified if the agent recalled Alice's preferences ---")


=== Thread 2: Testing memory recall (NEW conversation) ===


I remember that your name is Alice, you are a Senior Python developer, and you prefer concise, technical answers with code examples.


--- Memory persistence verified if the agent recalled Alice's preferences ---


## 16. Human-in-the-Loop (HITL) for Sensitive Operations

Add human oversight for sensitive tool operations. The agent will pause and wait for approval before executing configured tools.

**Decision Types:**
- `approve`: Execute the tool as proposed
- `edit`: Modify the tool arguments before execution  
- `reject`: Cancel the operation with feedback

**Architecture:**
```
Agent → Check Policy → [Interrupt] → Human Decision → Execute/Cancel
```


In [24]:
# Import Command for resuming after interrupts
from langgraph.types import Command

# Create agent with HITL for file write operations
# NOTE: create_deep_agent has a built-in `interrupt_on` parameter - no middleware needed!
hitl_agent = create_deep_agent(
    model=llm,
    tools=all_tools,
    system_prompt=enhanced_system_prompt,
    checkpointer=checkpointer,  # REQUIRED for HITL to work!
    store=persistent_store,
    backend=make_hybrid_backend,
    # Use the built-in interrupt_on parameter (NOT middleware!)
    interrupt_on={
        # Require approval for file writes (all decision types allowed)
        "write_file": True,
        "edit_file": True,
        # Allow approve/reject only (no editing) for task tool
        "task": {"allowed_decisions": ["approve", "reject"]},
        # NOTE: Tools not listed here are auto-approved (ls, read_file, etc.)
    }
)

print('[OK] HITL Agent created with approval requirements:')
print('  - write_file: Requires approval (approve/edit/reject)')
print('  - edit_file: Requires approval (approve/edit/reject)')
print('  - task (subagent): Requires approval (approve/reject only)')
print('  - All other tools: Auto-approved')


[OK] HITL Agent created with approval requirements:
  - write_file: Requires approval (approve/edit/reject)
  - edit_file: Requires approval (approve/edit/reject)
  - task (subagent): Requires approval (approve/reject only)
  - All other tools: Auto-approved


### Demo: Human-in-the-Loop Workflow

When the agent tries to write a file, execution pauses for human approval:


In [25]:
# Step 1: Request an action that requires approval
config = {"configurable": {"thread_id": "hitl-demo-v2"}}  # Fresh thread for new demo

# Collect interrupt events
interrupts = []
print("=== Requesting file write (will pause for approval) ===\n")

# CRITICAL: Use stream_mode="values" to properly receive interrupt state
# This is required by LangGraph to surface __interrupt__ in the step dictionary
for step in hitl_agent.stream(
    {"messages": [{"role": "user", "content": "Write a file at /memories/demo.md with the content: HITL demo successful"}]},
    config,
    stream_mode="values",  # REQUIRED for interrupt detection!
):
    # Check for messages in the step
    if "messages" in step:
        last_msg = step["messages"][-1]
        if hasattr(last_msg, 'content'):
            content_preview = extract_text_content(last_msg.content)[:200]
            print(f"Agent: {content_preview}...")
    
    # Check for interrupts - this is how LangGraph surfaces them
    if "__interrupt__" in step:
        print("\n" + "="*60)
        print("[INTERRUPT DETECTED]")
        print("="*60)
        for interrupt in step["__interrupt__"]:
            interrupts.append(interrupt)
            print(f"[INTERRUPT] ID: {interrupt.id}")
            # Extract action requests from the interrupt
            if hasattr(interrupt, 'value') and isinstance(interrupt.value, dict):
                action_requests = interrupt.value.get("action_requests", [])
                for request in action_requests:
                    description = request.get("description", "No description")
                    print(f"[INTERRUPT] {description}")
            print("[INTERRUPT] Awaiting human decision...")
        print("="*60)

if interrupts:
    print(f"\n[OK] Found {len(interrupts)} interrupt(s) requiring approval")
else:
    print("\n[INFO] No interrupts - operation completed without requiring approval")


=== Requesting file write (will pause for approval) ===

Agent: Write a file at /memories/demo.md with the content: HITL demo successful...
Agent: Write a file at /memories/demo.md with the content: HITL demo successful...
Agent: ...
Agent: ...

[INTERRUPT DETECTED]
[INTERRUPT] ID: 802244a332e0dcf32049c19736722573
[INTERRUPT] Tool execution requires approval

Tool: write_file
Args: {'file_path': '/memories/demo.md', 'content': 'HITL demo successful'}
[INTERRUPT] Awaiting human decision...

[OK] Found 1 interrupt(s) requiring approval


In [26]:
# Step 2: Resume with approval (run this after the interrupt above)
if interrupts:
    print("=== Approving the pending operation ===\n")
    
    # Create approval decisions for each interrupt
    decisions = [{"type": "approve"} for _ in interrupts]
    
    # Resume execution with the approval decisions
    # CRITICAL: Use Command with resume parameter to continue interrupted graph
    for step in hitl_agent.stream(
        Command(resume={"decisions": decisions}),
        config,  # Same thread_id to resume the interrupted execution
        stream_mode="values",
    ):
        if "messages" in step:
            last_msg = step["messages"][-1]
            if hasattr(last_msg, 'content'):
                content_preview = extract_text_content(last_msg.content)[:200]
                print(f"Agent: {content_preview}...")
        
        # Check for any new interrupts during resume
        if "__interrupt__" in step:
            print("[INFO] Additional interrupt detected during resume")
            for interrupt in step["__interrupt__"]:
                print(f"  ID: {interrupt.id}")
    
    print("\n[OK] Operation completed after approval")
else:
    print("[INFO] No pending interrupts to approve")


=== Approving the pending operation ===

Agent: ...
Agent: ...
Agent: Updated file /demo.md...
Agent: I have successfully written the file `/memories/demo.md` with the content "HITL demo successful"....

[OK] Operation completed after approval


## 17. FilesystemBackend: Local Disk Access (Optional)

Give the agent direct access to your local filesystem. This is powerful but requires caution.

**Warning**: Only use this in controlled environments. The agent will have read/write access to the specified root directory!


In [35]:
from deepagents.backends import FilesystemBackend
import tempfile
import os

# Create a safe sandbox directory for demo
sandbox_dir = tempfile.mkdtemp(prefix="deepagent_sandbox_")
print(f"[OK] Created sandbox directory: {sandbox_dir}")

# Agent with local filesystem access (sandboxed)
def make_local_backend(runtime):
    """
    Backend with local disk access.
    Combines:
    - /local/ -> Real filesystem (sandboxed with virtual_mode)
    - /memories/ -> Persistent store
    - Default -> Ephemeral state
    """
    return CompositeBackend(
        default=StateBackend(runtime),
        routes={
            "/memories/": StoreBackend(runtime),
            # CRITICAL: virtual_mode=True ensures paths are resolved relative to root_dir
            # Without this, paths like /hello_world.txt would resolve to C:\hello_world.txt on Windows
            "/local/": FilesystemBackend(root_dir=sandbox_dir, virtual_mode=True)
        }
    )

# Create filesystem-enabled agent
fs_agent = create_deep_agent(
    model=llm,
    tools=all_tools,
    system_prompt="""\
You are a research assistant with access to:
1. RAG knowledge base (search_knowledge_base)
2. MCP external tools
3. File system with three areas:
   - /local/ - Real files on the local machine (sandboxed)
   - /memories/ - Persistent cross-thread storage
   - /workspace/ - Temporary per-thread storage

Use /local/ for creating real files that persist on disk.
""",
    checkpointer=checkpointer,
    store=persistent_store,
    backend=make_local_backend,
    # Use built-in interrupt_on parameter (NOT middleware!)
    interrupt_on={
        "write_file": True,  # Requires approval
        "edit_file": True,   # Requires approval
    }
)

print('[OK] Filesystem Agent created:')
print(f'  - /local/* -> {sandbox_dir}')
print('  - /memories/* -> Persistent store')
print('  - All writes require HITL approval')


[OK] Created sandbox directory: C:\Users\pogawal\AppData\Local\Temp\deepagent_sandbox_x7pdmnlv
[OK] Filesystem Agent created:
  - /local/* -> C:\Users\pogawal\AppData\Local\Temp\deepagent_sandbox_x7pdmnlv
  - /memories/* -> Persistent store
  - All writes require HITL approval


### Demo: Filesystem Backend in Action

Let's test the filesystem-enabled agent by asking it to create a real file on disk.
The file will be created in the sandboxed directory and will persist after the notebook closes.

**IMPORTANT**: Before running the cells below:
1. **Re-run Cell 47 above** to create a fresh `fs_agent` with the `virtual_mode=True` fix
2. This will create a new sandbox directory and configure the agent correctly

**Note**: This demo uses HITL, so you'll need to approve the write operation.


In [36]:
# Demo: Create a real file using the filesystem backend
# This file will persist on the actual disk in the sandbox directory

from datetime import datetime
import os

# IMPORTANT: Verify the sandbox directory exists (Cell 47 must be run first!)
if 'sandbox_dir' not in dir() or not os.path.exists(sandbox_dir):
    print("[ERROR] sandbox_dir not defined or doesn't exist!")
    print("[ACTION REQUIRED] Please re-run Cell 47 (FilesystemBackend setup) first!")
    raise RuntimeError("Re-run Cell 47 first to create fs_agent with virtual_mode=True")

# Verify fs_agent exists
if 'fs_agent' not in dir():
    print("[ERROR] fs_agent not defined!")
    print("[ACTION REQUIRED] Please re-run Cell 47 (FilesystemBackend setup) first!")
    raise RuntimeError("Re-run Cell 47 first")

config = {"configurable": {"thread_id": f"filesystem-demo-{datetime.now().strftime('%H%M%S')}"}}  # Fresh thread with timestamp

# Get the actual current timestamp to include in the file
current_timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Collect interrupt events
fs_interrupts = []
print("=== Filesystem Backend Demo ===\n")
print(f"Sandbox directory: {sandbox_dir}")
print(f"Current timestamp: {current_timestamp}\n")

# Ask the agent to create a real file with the actual timestamp
file_content = f"""# Hello from Deep Agent!

This file was created by a LangGraph Deep Agent with FilesystemBackend.
It demonstrates direct access to the local filesystem (sandboxed for safety).

Created at: {current_timestamp}
Agent: Deep Agent with Gemini 2.5 Flash
"""

for step in fs_agent.stream(
    {"messages": [{"role": "user", "content": f"Create a file at /local/hello_world.txt with the following exact content:\n\n{file_content}"}]},
    config,
    stream_mode="values",
):
    # Check for messages
    if "messages" in step:
        last_msg = step["messages"][-1]
        if hasattr(last_msg, 'content'):
            content_preview = extract_text_content(last_msg.content)[:150]
            print(f"Agent: {content_preview}...")
    
    # Check for interrupts
    if "__interrupt__" in step:
        print("\n" + "="*60)
        print("[FILESYSTEM WRITE INTERRUPT]")
        print("="*60)
        for interrupt in step["__interrupt__"]:
            fs_interrupts.append(interrupt)
            print(f"[INTERRUPT] ID: {interrupt.id}")
            if hasattr(interrupt, 'value') and isinstance(interrupt.value, dict):
                action_requests = interrupt.value.get("action_requests", [])
                for request in action_requests:
                    description = request.get("description", "Write operation pending")
                    print(f"[INTERRUPT] {description}")
            print("[INTERRUPT] Awaiting approval to write to local disk...")
        print("="*60)

if fs_interrupts:
    print(f"\n[OK] Found {len(fs_interrupts)} interrupt(s) requiring approval")
else:
    print("\n[INFO] No interrupts - file may have been created automatically")


=== Filesystem Backend Demo ===

Sandbox directory: C:\Users\pogawal\AppData\Local\Temp\deepagent_sandbox_x7pdmnlv
Current timestamp: 2025-12-18 13:49:19

Agent: Create a file at /local/hello_world.txt with the following exact content:

# Hello from Deep Agent!

This file was created by a LangGraph Deep Agent w...
Agent: Create a file at /local/hello_world.txt with the following exact content:

# Hello from Deep Agent!

This file was created by a LangGraph Deep Agent w...
Agent: ...
Agent: ...

[FILESYSTEM WRITE INTERRUPT]
[INTERRUPT] ID: 5877d2f77fc6abac622dc231b8af92e8
[INTERRUPT] Tool execution requires approval

Tool: write_file
Args: {'file_path': '/local/hello_world.txt', 'content': '# Hello from Deep Agent!\n\nThis file was created by a LangGraph Deep Agent with FilesystemBackend.\nIt demonstrates direct access to the local filesystem (sandboxed for safety).\n\nCreated at: 2025-12-18 13:49:19\nAgent: Deep Agent with Gemini 2.5 Flash'}
[INTERRUPT] Awaiting approval to write to lo

In [37]:
# Approve the filesystem write and verify the file was created
if fs_interrupts:
    print("=== Approving filesystem write ===\n")
    
    decisions = [{"type": "approve"} for _ in fs_interrupts]
    
    for step in fs_agent.stream(
        Command(resume={"decisions": decisions}),
        config,
        stream_mode="values",
    ):
        if "messages" in step:
            last_msg = step["messages"][-1]
            if hasattr(last_msg, 'content'):
                content_preview = extract_text_content(last_msg.content)[:200]
                print(f"Agent: {content_preview}...")
    
    print("\n[OK] Filesystem write approved")
    
    # Verify the file exists on disk
    import os
    expected_file = os.path.join(sandbox_dir, "hello_world.txt")
    
    print(f"\n=== Verifying file on disk ===")
    print(f"Expected path: {expected_file}")
    
    if os.path.exists(expected_file):
        print("[OK] File exists on disk!")
        print("\nFile contents:")
        print("-" * 40)
        with open(expected_file, 'r') as f:
            print(f.read())
        print("-" * 40)
    else:
        print("[X] File not found - checking sandbox directory contents:")
        for item in os.listdir(sandbox_dir):
            print(f"  - {item}")
else:
    print("[INFO] No pending filesystem interrupts to approve")


=== Approving filesystem write ===

Agent: ...
Agent: ...
Agent: Updated file /hello_world.txt...
Agent: The file `/local/hello_world.txt` has been successfully created with the specified content....

[OK] Filesystem write approved

=== Verifying file on disk ===
Expected path: C:\Users\pogawal\AppData\Local\Temp\deepagent_sandbox_x7pdmnlv\hello_world.txt
[OK] File exists on disk!

File contents:
----------------------------------------
# Hello from Deep Agent!

This file was created by a LangGraph Deep Agent with FilesystemBackend.
It demonstrates direct access to the local filesystem (sandboxed for safety).

Created at: 2025-12-18 13:49:19
Agent: Deep Agent with Gemini 2.5 Flash
----------------------------------------


In [ ]:
# List files in the sandbox directory using the agent
# This demonstrates read operations (which don't require HITL approval)

config_read = {"configurable": {"thread_id": "filesystem-read-demo-v3"}}

print("=== Listing files via agent (no approval needed for reads) ===\n")

for step in fs_agent.stream(
    {"messages": [{"role": "user", "content": "List all files in /local/ and show me what's in the hello_world.txt file if it exists."}]},
    config_read,
    stream_mode="values",
):
    if "messages" in step:
        last_msg = step["messages"][-1]
        if hasattr(last_msg, 'content'):
            content = extract_text_content(last_msg.content)
            # Show more content for this read demo
            if len(content) > 50:  # Only show substantial responses
                print(f"Agent:\n{content}\n")

print("\n=== Filesystem demo complete ===")


In [ ]:
# Optional: Cleanup the sandbox directory
# Uncomment to remove the sandbox and its contents after the demo

# import shutil
# if os.path.exists(sandbox_dir):
#     shutil.rmtree(sandbox_dir)
#     print(f"[OK] Sandbox directory cleaned up: {sandbox_dir}")
# else:
#     print(f"[INFO] Sandbox directory already removed")

# For now, just show where the files are persisted
print(f"[INFO] Files persist at: {sandbox_dir}")
print("[INFO] To clean up manually, delete the sandbox directory or restart your computer.")


## 18. Subagents: Specialized Multi-Agent Architecture

Deep Agents can spawn **subagents** to handle specialized tasks. This enables:
- **Task Isolation**: Each subagent works in its own context
- **Specialized Tools**: Different agents use different tool sets
- **Parallel Execution**: Multiple subagents can work simultaneously
- **Context Management**: Prevents context window overflow

### Architecture
```
                    +-------------------+
                    |  Coordinator      |
                    |  (Main Agent)     |
                    +-------------------+
                           |
          +----------------+----------------+
          |                |                |
+---------v-----+  +-------v------+  +------v-------+
| Research      |  | Finance      |  | Weather      |
| Subagent      |  | Subagent     |  | Subagent     |
| (RAG + News)  |  | (Stocks)     |  | (Weather)    |
+---------------+  +--------------+  +--------------+
```

**Prerequisites**: Start the MCP servers before running this section:
```bash
# Terminal 1: Finance server
cd utils/mcp/fastmcp && python finance_server.py

# Terminal 2: News server  
cd utils/mcp/fastmcp && python news_server.py

# Terminal 3: Calculator server
cd utils/mcp/fastmcp && python calculator_server.py
```


In [41]:
# Connect to specialized MCP servers for subagent tools
# IMPORTANT: Start the servers first (see instructions above)

async def get_specialized_mcp_tools():
    """Connect to specialized MCP servers and organize tools by category."""
    tools_by_category = {}
    
    server_configs = {
        "finance": {"transport": "streamable_http", "url": "http://localhost:8001/mcp"},
        "news": {"transport": "streamable_http", "url": "http://localhost:8002/mcp"},
        "calculator": {"transport": "streamable_http", "url": "http://localhost:8003/mcp"},
        "weather": {"transport": "streamable_http", "url": "http://localhost:8000/mcp"},
    }
    
    for category, config in server_configs.items():
        try:
            client = MultiServerMCPClient({category: config})
            tools = await client.get_tools()
            if tools:
                tools_by_category[category] = tools
                print(f"[OK] {category.title()}: {len(tools)} tools")
                for t in tools:
                    print(f"     - {t.name}")
        except Exception as e:
            print(f"[X] {category.title()}: Not available")
            tools_by_category[category] = []
    
    return tools_by_category

print("=== Connecting to Specialized MCP Servers ===\n")
specialized_tools = await get_specialized_mcp_tools()
print(f"\n[OK] Connected to available servers")


=== Connecting to Specialized MCP Servers ===

[OK] Finance: 4 tools
     - get_stock_quote
     - get_company_info
     - get_stock_history
     - calculate_investment_return
[OK] News: 4 tools
     - get_top_headlines
     - search_news
     - search_wikipedia
     - get_current_datetime
[OK] Calculator: 3 tools
     - calculate_expression
     - convert_units
     - calculate_statistics
[OK] Weather: 1 tools
     - get_weather

[OK] Connected to available servers


### Create Specialized Subagents

Each subagent has:
1. **Specialized tools** - Only the tools relevant to its domain
2. **Focused system prompt** - Clear instructions for its specific role
3. **Independent context** - Isolated execution environment


In [42]:
# Define specialized system prompts for each subagent

RESEARCH_AGENT_PROMPT = """You are a Research Specialist subagent.

Your tools:
- search_knowledge_base: Query internal documentation
- search_wikipedia: Get encyclopedia information
- get_top_headlines: Get current news headlines

Your task: Research topics thoroughly using your tools.
Be comprehensive but concise. Cite your sources.
Return findings in a structured format."""

FINANCE_AGENT_PROMPT = """You are a Finance Analyst subagent.

Your tools:
- get_stock_quote: Get current stock prices
- get_company_info: Get company information
- calculate_investment_return: Calculate investment returns

Your task: Analyze financial data and provide insights.
Be precise with numbers. Always include source data.
Warn about investment risks when appropriate."""

WEATHER_AGENT_PROMPT = """You are a Weather & Time Specialist subagent.

Your tools:
- get_weather: Get weather for any location
- get_current_datetime: Get current date/time in any timezone
- convert_units: Convert temperature and other units

Your task: Provide weather information and time data.
Include both metric and imperial units when relevant.
Suggest appropriate clothing or activities based on weather."""

print("[OK] Subagent prompts defined")


[OK] Subagent prompts defined


In [43]:
# Create specialized subagents with their respective tools

# Collect tools for each subagent
research_tools = [search_knowledge_base]  # RAG tool
if specialized_tools.get("news"):
    research_tools.extend(specialized_tools["news"])

finance_tools = []
if specialized_tools.get("finance"):
    finance_tools.extend(specialized_tools["finance"])

weather_tools = []
if specialized_tools.get("weather"):
    weather_tools.extend(specialized_tools["weather"])
if specialized_tools.get("calculator"):
    weather_tools.extend(specialized_tools["calculator"])

# Create subagents
subagent_checkpointer = MemorySaver()

research_agent = create_deep_agent(
    model=llm,
    tools=research_tools,
    system_prompt=RESEARCH_AGENT_PROMPT,
    checkpointer=subagent_checkpointer,
) if research_tools else None

finance_agent = create_deep_agent(
    model=llm,
    tools=finance_tools,
    system_prompt=FINANCE_AGENT_PROMPT,
    checkpointer=subagent_checkpointer,
) if finance_tools else None

weather_agent = create_deep_agent(
    model=llm,
    tools=weather_tools,
    system_prompt=WEATHER_AGENT_PROMPT,
    checkpointer=subagent_checkpointer,
) if weather_tools else None

print("[OK] Subagents created:")
print(f"  - Research Agent: {len(research_tools)} tools")
print(f"  - Finance Agent: {len(finance_tools)} tools")
print(f"  - Weather Agent: {len(weather_tools)} tools")


[OK] Subagents created:
  - Research Agent: 5 tools
  - Finance Agent: 4 tools
  - Weather Agent: 4 tools


### Create Coordinator Agent

The coordinator uses custom tools to delegate work to subagents.
It analyzes user requests and routes them to the appropriate specialist.


In [44]:
# Create delegation tools for the coordinator agent

@tool
async def delegate_to_research(query: str) -> str:
    """
    Delegate a research task to the Research Specialist subagent.
    Use for: documentation queries, news, wikipedia lookups.
    
    Args:
        query: The research question or topic to investigate
    """
    if not research_agent:
        return "Research agent not available (missing tools)"
    
    response = await research_agent.ainvoke(
        {"messages": [{"role": "user", "content": query}]},
        {"configurable": {"thread_id": f"research-{hash(query) % 10000}"}}
    )
    return extract_text_content(response["messages"][-1].content)


@tool
async def delegate_to_finance(query: str) -> str:
    """
    Delegate a financial analysis task to the Finance Analyst subagent.
    Use for: stock prices, company info, investment calculations.
    
    Args:
        query: The financial question or analysis request
    """
    if not finance_agent:
        return "Finance agent not available (start finance_server.py first)"
    
    response = await finance_agent.ainvoke(
        {"messages": [{"role": "user", "content": query}]},
        {"configurable": {"thread_id": f"finance-{hash(query) % 10000}"}}
    )
    return extract_text_content(response["messages"][-1].content)


@tool
async def delegate_to_weather(query: str) -> str:
    """
    Delegate a weather/time task to the Weather Specialist subagent.
    Use for: weather forecasts, time zones, unit conversions.
    
    Args:
        query: The weather or time-related question
    """
    if not weather_agent:
        return "Weather agent not available (start weather_server.py first)"
    
    response = await weather_agent.ainvoke(
        {"messages": [{"role": "user", "content": query}]},
        {"configurable": {"thread_id": f"weather-{hash(query) % 10000}"}}
    )
    return extract_text_content(response["messages"][-1].content)

print("[OK] Delegation tools created")


[OK] Delegation tools created


In [45]:
# Create the Coordinator Agent

COORDINATOR_PROMPT = """You are a Coordinator Agent managing a team of specialized subagents.

Your team:
1. **Research Specialist** (delegate_to_research): Documentation, news, Wikipedia
2. **Finance Analyst** (delegate_to_finance): Stock prices, company info, investments
3. **Weather Specialist** (delegate_to_weather): Weather, time zones, conversions

Your role:
- Analyze user requests and determine which specialist(s) to involve
- Delegate tasks using the appropriate delegation tool
- Synthesize results from multiple specialists if needed
- Present a unified, coherent response to the user

Guidelines:
- For complex requests, break them down and delegate to multiple specialists
- Always explain which specialist handled each part of the request
- If a specialist is unavailable, inform the user gracefully"""

coordinator_tools = [delegate_to_research, delegate_to_finance, delegate_to_weather]

coordinator_agent = create_deep_agent(
    model=llm,
    tools=coordinator_tools,
    system_prompt=COORDINATOR_PROMPT,
    checkpointer=MemorySaver(),
)

print("[OK] Coordinator Agent created with 3 delegation tools")


[OK] Coordinator Agent created with 3 delegation tools


### Demo: Multi-Agent Coordination

Let's see the coordinator agent delegate tasks to specialists.


In [46]:
# Helper function for coordinator agent
async def ask_coordinator(question: str, thread_id: str = "coordinator-demo") -> str:
    """Send a question to the coordinator agent."""
    response = await coordinator_agent.ainvoke(
        {"messages": [{"role": "user", "content": question}]},
        {"configurable": {"thread_id": thread_id}}
    )
    return extract_text_content(response["messages"][-1].content)

print("[OK] Coordinator helper ready")


[OK] Coordinator helper ready


In [ ]:
# Demo 1: Single specialist delegation (Research)
print("=== Demo 1: Research Specialist ===\n")

response1 = await ask_coordinator(
    "What is the Model Context Protocol (MCP)? Search the knowledge base.",
    thread_id="demo-research"
)
display_response(response1)


In [ ]:
# Demo 2: Single specialist delegation (Finance)
# NOTE: Requires finance_server.py running on port 8001
print("=== Demo 2: Finance Specialist ===\n")

response2 = await ask_coordinator(
    "Get me the current stock price for NVIDIA (NVDA) and Apple (AAPL).",
    thread_id="demo-finance"
)
display_response(response2)


In [ ]:
# Demo 3: Single specialist delegation (Weather)
# NOTE: Requires weather_server.py running on port 8000
print("=== Demo 3: Weather Specialist ===\n")

response3 = await ask_coordinator(
    "What's the weather in Tokyo and what time is it there?",
    thread_id="demo-weather"
)
display_response(response3)


In [ ]:
# Demo 4: Multi-specialist coordination
# The coordinator should delegate to multiple specialists
print("=== Demo 4: Multi-Specialist Coordination ===\n")

response4 = await ask_coordinator(
    """I'm planning a business trip to London next week. Please help me with:
    1. What's the current weather like in London?
    2. Get me the stock price for British Airways (IAG.L)
    3. What do you know about deep agents from the knowledge base?
    
    Summarize all findings in a trip planning report.""",
    thread_id="demo-multi"
)
display_response(response4)


### Subagent Architecture Summary

**How it works:**
1. User sends request to **Coordinator Agent**
2. Coordinator analyzes request and identifies required specialists
3. Coordinator calls delegation tools (`delegate_to_research`, etc.)
4. Each delegation tool invokes the corresponding **Subagent**
5. Subagent uses its specialized MCP tools to gather information
6. Results flow back to Coordinator
7. Coordinator synthesizes and presents unified response

**Benefits:**
- **Specialization**: Each agent is optimized for its domain
- **Isolation**: Subagents work in separate contexts
- **Scalability**: Easy to add new specialists
- **Maintainability**: Clear separation of concerns

**Production considerations:**
- Use persistent checkpointers (PostgresSaver) for subagent memory
- Add HITL for sensitive subagent operations
- Implement retry logic and error handling
- Consider parallel execution for independent subagent tasks


### Optional: Test Subagents Directly

You can also interact with individual subagents directly for debugging or specialized tasks.


In [ ]:
# Direct subagent test - Research Agent
async def test_subagent(agent, name: str, query: str):
    """Test a subagent directly."""
    if not agent:
        print(f"[X] {name} not available")
        return
    
    print(f"=== Testing {name} ===\n")
    response = await agent.ainvoke(
        {"messages": [{"role": "user", "content": query}]},
        {"configurable": {"thread_id": f"test-{name.lower()}"}}
    )
    result = extract_text_content(response["messages"][-1].content)
    display_response(result)

# Test the research agent directly
await test_subagent(
    research_agent, 
    "Research Agent", 
    "Search the knowledge base for information about MCP middleware"
)


In [ ]:
# Test Finance Agent directly (requires finance_server.py on port 8001)
await test_subagent(
    finance_agent,
    "Finance Agent", 
    "Get the current stock price for Microsoft (MSFT)"
)


In [ ]:
# Test Weather Agent directly (requires weather_server.py on port 8000)
await test_subagent(
    weather_agent,
    "Weather Agent", 
    "What's the weather in New York and convert 72 Fahrenheit to Celsius"
)


### Starting the MCP Servers

Run these commands in separate terminals to start all MCP servers:

```powershell
# Terminal 1: Weather Server (port 8000)
cd C:\Users\pogawal\WorkFolder\Documents\Python\ai-dev-agent\utils\mcp\fastmcp
C:\App\Anaconda\python.exe weather_server.py

# Terminal 2: Finance Server (port 8001)
cd C:\Users\pogawal\WorkFolder\Documents\Python\ai-dev-agent\utils\mcp\fastmcp
C:\App\Anaconda\python.exe finance_server.py

# Terminal 3: News Server (port 8002)
cd C:\Users\pogawal\WorkFolder\Documents\Python\ai-dev-agent\utils\mcp\fastmcp
C:\App\Anaconda\python.exe news_server.py

# Terminal 4: Calculator Server (port 8003)
cd C:\Users\pogawal\WorkFolder\Documents\Python\ai-dev-agent\utils\mcp\fastmcp
C:\App\Anaconda\python.exe calculator_server.py
```

After starting the servers, re-run Cell 54 to connect to them.


## 19. Direct Tool Testing: News Server

Test the news server tools directly without LLM usage.
This imports the tool functions directly and calls them to verify the APIs work.

**Requirements:**
```bash
pip install newsapi-python wikipedia
```

**Environment:**
- `NEWS_API_KEY` must be set (get free key at https://newsapi.org)


In [1]:
# Direct import test - verify the news server module works without MCP
# This tests the underlying functions, not the MCP wrapper

import sys
sys.path.insert(0, r"C:\Users\pogawal\WorkFolder\Documents\Python\ai-dev-agent")

# Import the tools directly from the news server module
from utils.mcp.fastmcp.news_server import (
    get_top_headlines,
    search_news,
    search_wikipedia,
    get_wikipedia_page,
    get_current_datetime,
    get_news_sources
)

print("[OK] News server module imported successfully")
print("\nAvailable tools:")
print("  - get_top_headlines(category, country, query, page_size)")
print("  - search_news(query, page_size, sort_by, language)")
print("  - search_wikipedia(query, sentences)")
print("  - get_wikipedia_page(title)")
print("  - get_current_datetime(timezone)")
print("  - get_news_sources(category, country)")


[OK] News server module imported successfully

Available tools:
  - get_top_headlines(category, country, query, page_size)
  - search_news(query, page_size, sort_by, language)
  - search_wikipedia(query, sentences)
  - get_wikipedia_page(title)
  - get_current_datetime(timezone)
  - get_news_sources(category, country)


In [2]:
# Test 1: Date/Time (no API key required)
print("=== Test: get_current_datetime ===\n")

result = get_current_datetime("UTC")
print(result)
print()

result = get_current_datetime("CET")
print(result)


=== Test: get_current_datetime ===

Current Date/Time (UTC):
Date: Thursday, December 18, 2025
Time: 03:00:23 PM
ISO: 2025-12-18T15:00:23

Current Date/Time (CET):
Date: Thursday, December 18, 2025
Time: 04:00:23 PM
ISO: 2025-12-18T16:00:23


In [3]:
# Test 2: Wikipedia (no API key required)
print("=== Test: search_wikipedia ===\n")

result = search_wikipedia("Python programming language", sentences=3)
print(result)


=== Test: search_wikipedia ===

Wikipedia: Python (programming language)

Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation. Python is dynamically type-checked and garbage-collected.

Read more: https://en.wikipedia.org/wiki/Python_(programming_language)


In [4]:
# Test 3: Wikipedia page (no API key required)
print("=== Test: get_wikipedia_page ===\n")

result = get_wikipedia_page("Artificial intelligence")
print(result)


=== Test: get_wikipedia_page ===

Wikipedia: Artificial intelligence

Summary:
Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.
High-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); virtual assistants (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., language models and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: "A lot of cu

In [2]:
# Test 4: NewsAPI - Top Headlines (requires NEWS_API_KEY)
import os
print("=== Test: get_top_headlines ===\n")

if os.environ.get("NEWS_API_KEY"):
    result = get_top_headlines(category="technology", country="us", page_size=3)
    print(result)
else:
    print("[X] NEWS_API_KEY not set in environment")
    print("    Set it with: os.environ['NEWS_API_KEY'] = 'your-key'")


=== Test: get_top_headlines ===

Top Technology Headlines (US):

1. [IGN] Video Game Physical Software and Hardware Sales Just Had the Worst November in the U.S. Since 1995 - IGN
   Last month, November, was a shockingly terrible month for video game sales in the U.S. While we traditionally think of N...
   Published: 2025-12-17 14:11
   https://www.ign.com/articles/video-game-physical-software-and-hardware-sales-just-had-the-worst-november-in-the-us-since-1995

2. [Pinkbike.com] Product of the Year Nominees - Pinkbike
   The products that raised the bar the highest in their own categories.
   Published: 2025-12-17 12:00
   https://www.pinkbike.com/news/2025-pinkbike-awards-product-of-the-year-nominees.html

3. [IGN] Elden Ring: Nightreign Update 1.03.1 Is a Big One, Makes Key Balance Changes and Adds New Content for the Forsaken Hollows DLC - IGN
   None
   Published: 2025-12-17 11:44
   https://www.ign.com/articles/elden-ring-nightreign-update-1031-is-a-big-one-makes-key-balance-chan

In [3]:
# Test 5: NewsAPI - Search News (requires NEWS_API_KEY)
print("=== Test: search_news ===\n")

if os.environ.get("NEWS_API_KEY"):
    result = search_news(query="artificial intelligence", page_size=3, sort_by="publishedAt")
    print(result)
else:
    print("[X] NEWS_API_KEY not set - skipping")


=== Test: search_news ===

News Search: 'artificial intelligence' (15326 total results, showing 3)

1. [National Institutes of Health] Request for Information: Call for Input into NIH's Best Pharmaceuticals for Children Act (BPCA) Priorities for 2026 and Beyond
   NIH Funding Opportunities and Notices in the NIH Guide for Grants and Contracts: Request for Information: Call for Input into NIH's Best Pharmaceutica...
   Published: 2025-12-17 15:10
   https://grants.nih.gov/grants/guide/notice-files/NOT-HD-25-004.html

2. [Business Insider] How AI is inspiring companies to adopt skills-based hiring
   AI is accelerating the shift toward skills-based hiring, and in the process reshaping how companies go about recruiting.
   Published: 2025-12-17 15:10
   https://www.businessinsider.com/ai-accelerating-trend-job-hires-college-degrees-matter-less-2025-12

3. [MetroWest Daily News] Hudson MA High School placed in lockdown. What authorities are saying
   Hudson High School was placed on lockdo

In [4]:
# Test 6: NewsAPI - Get Sources (requires NEWS_API_KEY)
print("=== Test: get_news_sources ===\n")

if os.environ.get("NEWS_API_KEY"):
    result = get_news_sources(category="technology")
    print(result)
else:
    print("[X] NEWS_API_KEY not set - skipping")

print("\n" + "="*50)
print("[OK] News server testing complete!")
print("="*50)


=== Test: get_news_sources ===

News Sources:

- Ars Technica (technology)
  The PC enthusiast's resource. Power users and the tools they love, without compu...
  https://arstechnica.com

- Crypto Coins News (technology)
  Providing breaking cryptocurrency news - focusing on Bitcoin, Ethereum, ICOs, bl...
  https://www.ccn.com

- Engadget (technology)
  Engadget is a web magazine with obsessive daily coverage of everything new in ga...
  https://www.engadget.com

- Gruenderszene (technology)
  Online-Magazin für Startups und die digitale Wirtschaft. News und Hintergründe z...
  http://www.gruenderszene.de

- Hacker News (technology)
  Hacker News is a social news website focusing on computer science and entreprene...
  https://news.ycombinator.com

- Recode (technology)
  Get the latest independent tech news, reviews and analysis from Recode with the ...
  http://www.recode.net

- T3n (technology)
  Das Online-Magazin bietet Artikel zu den Themen E-Business, Social Media, Startu...
  h